<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/rtgender/RtGender_Post_%26_Responses_BoW_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# install datasets
!pip install datasets
!pip install keras-tuner

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline
import seaborn as sns

from collections import Counter
from IPython.display import display, HTML
from datasets import ClassLabel


import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout
from keras.models import Sequential
from tensorflow.keras.layers import TextVectorization
import tensorflow.keras.backend as K
# for hyperparameter tunning
import keras_tuner as kt
from keras_tuner import HyperModel

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences


# import sklearn to calculate the metrics
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
print(tf.__version__)

2.6.0


In [5]:
import os
os.getcwd()

'/content'

In [6]:
#mount from Google Drive 

try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


# Split Data

In [6]:
# from datasets import load_dataset, DatasetDict, table
# posts = load_dataset('peixian/rtGender', 'posts', split = 'train')
# posts_df = posts.to_pandas()

# from datasets import load_dataset, DatasetDict, table
# responses = load_dataset('peixian/rtGender', 'responses', split = 'train')
# responses_df = responses.to_pandas()

Downloading:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.67G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset rt_gender downloaded and prepared to /root/.cache/huggingface/datasets/rt_gender/responses/1.1.0/62d8f34778a4eda10ed650d804cbb7ad4da391ac28f38034f424a2e9a7fc43d1. Subsequent calls will reuse this data.


In [10]:
# import csv
# ## ids to split posts and responses

# with open('/content/drive/MyDrive/w266/post_ids_train', newline='') as f:
#     reader = csv.reader(f)
#     post_id_train_list = list(reader)    
#     # unnest doubled list    
#     post_id_train_list = post_id_train_list[0]   

# with open('/content/drive/MyDrive/w266/post_ids_dev', newline='') as f:
#     reader = csv.reader(f)
#     post_id_dev_list = list(reader)    
#     post_id_dev_list = post_id_dev_list[0]    

# with open('/content/drive/MyDrive/w266/post_ids_test', newline='') as f:
#     reader = csv.reader(f)
#     post_id_test_list = list(reader) 
#     post_id_test_list = post_id_test_list[0]
    

In [12]:
# # split dataframes based on post_ids
# responses_train = responses_df[responses_df.post_id.isin(post_id_train_list)]
# responses_train.to_parquet('/content/drive/MyDrive/w266/responses_train.parquet.gzip',
#               compression='gzip') 

# responses_dev = responses_df[responses_df.post_id.isin(post_id_dev_list)]
# responses_dev.to_parquet('/content/drive/MyDrive/w266/responses_dev.parquet.gzip',
#               compression='gzip') 

# responses_test = responses_df[responses_df.post_id.isin(post_id_test_list)]
# responses_test.to_parquet('/content/drive/MyDrive/w266/responses_tests.parquet.gzip',
#               compression='gzip') 


# Load GlOve

In [7]:
# load pre-trained word embeddings. In this case Glove
# This is commented out to avoid downloading it again
!wget http://nlp.stanford.edu/data/glove.6B.zip -P ~/data/

# unzip the file
# commented out for the same reason above#
!unzip ~/data/glove.6B.zip -d ~/data/

--2021-09-30 20:17:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-30 20:17:24--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-30 20:17:24--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/root/data/glove.6B.zi

In [8]:
# path to glove file- will use the embeddings with dimension = 300
glove_file ="/root/data/glove.6B.300d.txt"

# embedding layer 
embeddings_index = {}
f = open(os.path.join(glove_file))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


# Helper functions

In [9]:
#Posts, Responses, & Annotations: op_gender (M:0, W:1)
gender_binary_mappings = {"M": 0, "W": 1}
gender_labels = ["M", "W"]

# Annotations sentiment
# group mixed and neutral 
sentiment_mappings = {'Positive': 2, 'Mixed': 1, 'Neutral': 1, 'Negative':0}
sentiment_labels = ['Positive', 'Mixed', 'Neutral', 'Negative']


In [10]:
def tokenize_Xtrain_and_Xdev(x_train, x_dev):
    '''Combine tokenize and split back the X_train and 
    X_dev or X_test datasets. Then create an embedding matrix.'''

    # note the length of the training index
    train_idx = len(x_train)
    #combine train and dev data and then tokenize
    texts = x_train.append(x_dev)

    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    print('Train + Dev %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=max_sequence_length)
    print('Shape of data tensor:', data.shape)
    
    x_train = data[:train_idx]
    x_dev = data[train_idx:]

    print('Shape of train data tensor:', x_train.shape)
    print('Shape of dev data tensor:', x_dev.shape)
    print("-"*60)
    
    # print top 5 most and least common tokens
    print("top 5 most common tokens: ", sorted(word_index, key=word_index.get, reverse=True)[-5:])
    print("top 5 least common tokens: ", sorted(word_index, key=word_index.get, reverse=True)[:5])
    print("-"*60)

    
    # build embedding matrix to use it in the model
    dimensions_emb = 300

    # zero position is zero by default in keras
    embedding_matrix = np.zeros((len(word_index) + 1, dimensions_emb)) 

    total_tokens = len(word_index) + 1
    with_embedding = []
    without_embedding = []

    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            with_embedding.append(word)
        else:
            without_embedding.append(word)

    print(f'Number of words with embeddings found: {len(with_embedding)}')
    print(f'Number of words with embeddings NOT found: {len(without_embedding)}')        

    
    return x_train, x_dev, embedding_matrix


In [11]:
def binary_loss_accuracy_plots(training_history):
    '''
    Returns plots for loss and accuracy during the training process of a NN.
    Parameters:
    training_history: object that stores the training history of the NN (from model.fit(...))
    '''
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    # summarize history for accuracy
    ax1.plot(training_history.history['binary_accuracy'])
    ax1.plot(training_history.history['val_binary_accuracy'])
    ax1.set_title('model accuracy')
    ax1.set_ylabel('accuracy')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'dev'], loc='upper left')

    # summarize history for loss
    ax2.plot(training_history.history['loss'])
    ax2.plot(training_history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'dev'], loc='upper left')

    plt.show()

def multiclass_loss_accuracy_plots(training_history):
    '''
    Returns plots for loss and accuracy during the training process of a NN.
    Parameters:
    training_history: object that stores the training history of the NN (from model.fit(...))
    '''
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    # summarize history for accuracy
    ax1.plot(training_history.history['categorical_accuracy'])
    ax1.plot(training_history.history['val_categorical_accuracy'])
    ax1.set_title('model accuracy')
    ax1.set_ylabel('accuracy')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'dev'], loc='upper left')

    # summarize history for loss
    ax2.plot(training_history.history['loss'])
    ax2.plot(training_history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'dev'], loc='upper left')

    plt.show()
    

In [12]:
def confusion_matrix(x_dev, original_dev_labels, class_labels):

    # identify the correct class
    max_class_idx = np.argmax(y_pred, axis = 1)
    
    # Create a confusion matrix
    cm = tf.math.confusion_matrix(original_dev_labels, max_class_idx)
    cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

    sns.heatmap(
        cm, annot=True,
        xticklabels=class_labels,
        yticklabels=class_labels,
        fmt='.2%', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
    
    # print classification report
    print(metrics.classification_report(original_dev_labels, max_class_idx))
    # print global weighted f-1 score
    f1 = metrics.f1_score(original_dev_labels, max_class_idx, pos_label=1, average='weighted')*100
    print(f"Weighted f1-score: %.2f%%" %f1)


In [13]:
def binary_model_builder(hp):
    model = tf.keras.Sequential()
    model.add(embedding_layer) 
    model.add(tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1))) #avg

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 10 and 100
    hp_units = hp.Int('units', min_value=10, max_value=100, step=20)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune dropout layer 
    dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, default=0.0, step=0.1)
    model.add(Dropout(dropout))
        
    # output layer 
    model.add(Dense(2, activation='softmax'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy', 
                metrics=['binary_accuracy', 'binary_crossentropy'])

    return model

# Posts Op Gender

In [19]:
# Load post data
posts_train = pd.read_parquet('/content/drive/MyDrive/w266/posts_train.parquet.gzip')
posts_test = pd.read_parquet('/content/drive/MyDrive/w266/posts_tests.parquet.gzip')
posts_dev= pd.read_parquet('/content/drive/MyDrive/w266/posts_dev.parquet.gzip')

In [20]:
X_train = posts_train['post_text']
y_train = posts_train['op_gender'].map(gender_binary_mappings)
X_dev = posts_dev['post_text']
y_dev = posts_dev['op_gender'].map(gender_binary_mappings)

In [27]:
# tokenize data and return embeddings matrix 
max_tokens = 10000
# consider this maximum number of words
max_sequence_length = 100
tokenizer = Tokenizer(num_words=max_tokens)

x_train, x_dev, embedding_matrix = tokenize_Xtrain_and_Xdev(X_train, X_dev)

# one-hot encodign and reshape labels
print("-"*60)

train_labels = to_categorical(np.asarray(y_train))
dev_labels = to_categorical(np.asarray(y_dev))

print('Shape of train label tensor:', train_labels.shape)
print('Shape of dev label tensor:', dev_labels.shape)

Train + Dev 559520 unique tokens.
Shape of data tensor: (2883387, 100)
Shape of train data tensor: (2137983, 100)
Shape of dev data tensor: (745404, 100)
------------------------------------------------------------
top 5 most common tokens:  ['of', 'a', 'and', 'to', 'the']
top 5 least common tokens:  ['digikey', 'mu6300', 'aliant', 'metrotown', 'beamdog']
------------------------------------------------------------
Number of words with embeddings found: 146683
Number of words with embeddings NOT found: 412837
------------------------------------------------------------
Shape of train label tensor: (2137983, 2)
Shape of dev label tensor: (745404, 2)


In [28]:
# delete model if exists
try:
    del tuner
except:
    pass

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [29]:
def post_op_binary_model_builder(hp):
    model = tf.keras.Sequential()
    model.add(embedding_layer) 
    model.add(tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1))) #avg

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 10 and 25
    hp_units = hp.Int('units', min_value=10, max_value=25, step=5)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune dropout layer 
    dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, default=0.0, step=0.1)
    model.add(Dropout(dropout))
        
    # output layer 
    model.add(Dense(2, activation='sigmoid'))

    # Tune the learning rate for the optimizer
  
    model.compile(optimizer='adam',
                loss='binary_crossentropy', 
                metrics=['binary_accuracy', 'binary_crossentropy'])

    return model

In [31]:
tuner = kt.Hyperband(post_op_binary_model_builder,
                     objective='val_binary_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='my_dir',
                     project_name='op_gender')

tuner.search(x_train, train_labels, validation_data=(x_dev, dev_labels),
             epochs=1)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 18 Complete [00h 25m 00s]
val_binary_accuracy: 0.7101229429244995

Best val_binary_accuracy So Far: 0.717460572719574
Total elapsed time: 04h 26m 16s
INFO:tensorflow:Oracle triggered exit


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyError: ignored

In [32]:

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}.""")
model = tuner.hypermodel.build(best_hps)



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 25.


In [33]:
model.save('/content/drive/MyDrive/w266/posts_op_gender_baseline_model')

# fit model 
history = model.fit(x_train, train_labels,
                    validation_data=(x_dev, dev_labels),
                    epochs=5, verbose=0)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/w266/posts_op_gender_baseline_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/w266/posts_op_gender_baseline_model/assets


ValueError: ignored

In [ ]:
binary_loss_accuracy_plots(history)
y_pred = model.predict(x_dev) #predict
confusion_matrix(x_dev, y_dev, gender_labels)

# Responses: Respondent Gender

In [14]:
# Load responses data
responses_train = pd.read_parquet('/content/drive/MyDrive/w266/responses_train.parquet.gzip')
responses_dev= pd.read_parquet('/content/drive/MyDrive/w266/responses_dev.parquet.gzip')

X_train = responses_train['response_text']
y_train = responses_train['responder_gender'].map(gender_binary_mappings)
X_dev = responses_dev['response_text']
y_dev = responses_dev['responder_gender'].map(gender_binary_mappings)

In [ ]:
# tokenize data and return embeddings matrix 
max_tokens = 10000
# consider this maximum number of words
max_sequence_length = 100
from keras.preprocessing.text import Tokenizer
                                         
tokenizer = Tokenizer(num_words=max_tokens)

x_train, x_dev, embedding_matrix = tokenize_Xtrain_and_Xdev(X_train, X_dev)

# one-hot encodign and reshape labels
print("-"*60)

train_labels = to_categorical(np.asarray(y_train))
dev_labels = to_categorical(np.asarray(y_dev))

print('Shape of train label tensor:', train_labels.shape)
print('Shape of dev label tensor:', dev_labels.shape)

Train + Dev 2730176 unique tokens.


In [ ]:
# delete model if exists
try:
    del tuner
except:
    pass

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

def responses_binary_model_builder(hp):
    model = tf.keras.Sequential()
    model.add(embedding_layer) 
    model.add(tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1))) #avg

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 10 and 100
    hp_units = hp.Int('units', min_value=20, max_value=100, step=20)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune dropout layer 
    dropout = hp.Float('dropout', min_value=0.2, max_value=0.4, default=0.0, step=0.1)
    model.add(Dropout(dropout))
        
    # output layer 
    model.add(Dense(2, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
   # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer='adam',#tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy', 
                metrics=['binary_accuracy', 'binary_crossentropy'])

    return model

In [2]:
tuner = kt.Hyperband(responses_binary_model_builder,
                     objective='val_binary_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='my_dir',
                     project_name='respondent_gender')

tuner.search(x_train, train_labels, validation_data=(x_dev, dev_labels),
             epochs=1)

NameError: ignored

In [ ]:


# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}.
""")
model = tuner.hypermodel.build(best_hps)


In [ ]:
model.save('/content/drive/MyDrive/w266/responses_respondent_gender_baseline_model')

# fit model 
history = model.fit(x_train, train_labels,
                    validation_data=(x_dev, dev_labels),
                    epochs=5, verbose=0)